In [1]:
import numpy as np
import pandas as pd
import os
import math

In [2]:
cwd = os.getcwd()
print(cwd)

D:\Analytics


In [3]:
track_csv = r'D:\Analytics\fma_metadata\raw_tracks.csv'
genre_csv = r'D:\Analytics\fma_metadata\genres.csv'
tracks = pd.read_csv(track_csv)  # note: this will raise a low_memory warning from Pandas. This warning is meaningless, see: https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options
genre_csv = pd.read_csv(genre_csv)

In [4]:
tracks.head()

,track_id,album_id,album_title,album_url,artist_id,artist_name,artist_url,artist_website,license_image_file,license_image_file_large,...,track_information,track_instrumental,track_interest,track_language_code,track_listens,track_lyricist,track_number,track_publisher,track_title,track_url
0,2,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,4656,en,1293,NaN,3,NaN,Food,http://freemusicarchive.org/music/AWOL/AWOL_-_...
1,3,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1470,en,514,NaN,4,NaN,Electric Ave,http://freemusicarchive.org/music/AWOL/AWOL_-_...
2,5,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1933,en,1151,NaN,6,NaN,This World,http://freemusicarchive.org/music/AWOL/AWOL_-_...
3,10,6.0,Constant Hitmaker,http://freemusicarchive.org/music/Kurt_Vile/Co...,6,Kurt Vile,http://freemusicarchive.org/music/Kurt_Vile/,http://kurtvile.com,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,54881,en,50135,NaN,1,NaN,Freeway,http://freemusicarchive.org/music/Kurt_Vile/Co...
4,20,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,978,en,361,NaN,3,NaN,Spiritual Level,http://freemusicarchive.org/music/Chris_and_Ni...


In [5]:
tracks['track_genres'].head()

0    [{'genre_id': '21', 'genre_title': 'Hip-Hop', ...
1    [{'genre_id': '21', 'genre_title': 'Hip-Hop', ...
2    [{'genre_id': '21', 'genre_title': 'Hip-Hop', ...
3    [{'genre_id': '10', 'genre_title': 'Pop', 'gen...
4    [{'genre_id': '76', 'genre_title': 'Experiment...
Name: track_genres, dtype: object

In [6]:
g_ids = []
for genre in tracks['track_genres'].to_list():
    try:
        s = eval(genre)[0]  # this is pretty dangerous. use w/ caution
        g_ids.append(int(s['genre_id']))
    except:
        g_ids.append('error')

In [7]:
errors = [g for g in g_ids if g == 'error']
print(len(errors))  # these all have missing genre information. must drop

2609


In [8]:
# get only the top-level genres from genre_csv
top_genres = []
top_genres_id = []
genre_id = genre_csv['genre_id'].tolist()
title = genre_csv['title'].tolist()
top_level = genre_csv['top_level'].tolist()
for i in range(len(genre_id)):
    if genre_id[i] == top_level[i]:
        top_genres.append(title[i])
        top_genres_id.append(genre_id[i])
print(top_genres)
print(top_genres_id)
print(len(top_genres), len(top_genres_id))
genre_lut = dict(zip(top_genres_id, top_genres))

['International', 'Blues', 'Jazz', 'Classical', 'Old-Time / Historic', 'Country', 'Pop', 'Rock', 'Easy Listening', 'Soul-RnB', 'Electronic', 'Folk', 'Spoken', 'Hip-Hop', 'Experimental', 'Instrumental']
[2, 3, 4, 5, 8, 9, 10, 12, 13, 14, 15, 17, 20, 21, 38, 1235]
16 16


In [9]:
# match genre ids with top level genre names
g_top = []
for g in g_ids:
    if g != 'error':
        loc = genre_id.index(g)
        top = top_level[loc]
        top_name = genre_lut[top]
        g_top.append(top_name)
    else:
        g_top.append('missing')

In [10]:
cleaned_genre_dict = {'track id': tracks['track_id'].tolist(), 'genre': g_top}

In [11]:
cleaned_genre_output = pd.DataFrame(cleaned_genre_dict)

In [14]:
cleaned_genre_output.head()
cleaned_genre_output.genre.unique()

array(['Hip-Hop', 'Pop', 'Rock', 'Experimental', 'Folk', 'Jazz',
       'Electronic', 'Blues', 'missing', 'Spoken', 'International',
       'Soul-RnB', 'Classical', 'Country', 'Easy Listening',
       'Old-Time / Historic', 'Instrumental'], dtype=object)

In [100]:
cleaned_genre_output.to_csv(r'D:\Analytics\fma_clean_genres.csv')